# 量化 (Quantization)

量化是將模型權重和激活函數從浮點數轉換為較低精度（如 INT8）以減少模型大小和計算需求的方法。這可以顯著提高推理速度並減少記憶體使用量，特別是在資源受限的設備上，如移動設備或嵌入式系統。

## PyTorch 中的量化

PyTorch 提供了多種量化方法，包括靜態量化 (Static Quantization)、動態量化 (Dynamic Quantization) 和量化感知訓練 (Quantization Aware Training)。在這裡，我們將展示如何對 `torch.nn` 模型進行動態量化，並比較量化前後的速度。

### 1. 安裝必要的庫

```python
# 安裝 torch 庫
!pip install torch


In [10]:
import torch
import torch.nn as nn
import time


In [11]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleModel()

In [12]:
# 將模型轉換為動態量化模型
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)

In [14]:
# 創建測試數據
input_data = torch.randn(1000, 784)

In [16]:
# 測量原始模型的推理時間
start_time = time.time()
with torch.no_grad():
    for _ in range(100):
        model(input_data)
original_time = time.time() - start_time

# 測量量化模型的推理時間
start_time = time.time()
with torch.no_grad():
    for _ in range(100):
        quantized_model(input_data)
quantized_time = time.time() - start_time

# 顯示結果
print(f"原始模型推理時間: {original_time:.4f} 秒")
print(f"量化模型推理時間: {quantized_time:.4f} 秒")
print(f"速度提升: {original_time / quantized_time:.2f} 倍")

原始模型推理時間: 0.1810 秒
量化模型推理時間: 0.1500 秒
速度提升: 1.21 倍
